In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = load_model('/content/drive/MyDrive/models/model2_augmented_datacleared.keras')

In [ ]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 72, 33, 33, 4)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 72, 128)             │       1,142,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 72, 64)              │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 72, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ yield_output (Dense)                 │ (None, 72, 1)               │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,452,453 (13.17 MB)

 Trainable params: 1,150,753 (4.39 MB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 2,301,508 (8.78 MB)

# Satellite Data

In [ ]:
# Satellite data

import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array




# Define the paths
base_path = "/content/drive/MyDrive/GEE_Images/Seasonal_Images"
years = ["2019", "2020", "2021", "2022", "2023"]
seasons = ["Yala", "Maha"]

cnn_input_shape = (256, 256, 4)  # Adjust this based on your CNN model's input shape

# Function to load and preprocess images
def load_images_from_folder(folder_path, input_shape, expected_count):
    images = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.npy'):
            img_path = os.path.join(folder_path, filename)
            img = np.load(img_path)
            img = np.nan_to_num(img, nan=0.0)
            if img.shape != input_shape:
                print(f"Warning: Image {filename} has shape {img.shape}, expected {input_shape}")
                img = np.resize(img, input_shape)
            # Normalize the image
            img = img / np.max(img) if np.max(img) > 0 else img
            images.append(img)
    # Handle cases with missing images by padding or discarding
    if len(images) < expected_count:
        # Pad with zeros if fewer images than expected
        while len(images) < expected_count:
            padding_image = np.zeros(input_shape)
            images.append(padding_image)
    elif len(images) > expected_count:
        # Trim the excess images if more than expected
        images = images[:expected_count]

    return np.array(images)

# Function to prepare data for the model
def prepare_data_for_model(base_path, years, seasons, input_shape):
    all_season_data = []
    for year in years:
        for season in seasons:
            folder_path = os.path.join(base_path, year, season)
            expected_count = 6 if season == "Maha" else 6  # Adjust expected count based on season
            images = load_images_from_folder(folder_path, input_shape, expected_count)
            all_season_data.append(images)
    return all_season_data  # Return as a list of arrays instead of a single array

satellite_placeholder = np.zeros((6, 1024, 1024, 4))

# Prepare the satellite data
satellite_input = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Hambanthota/Seasonal_Images_Hambanthota", years, seasons, cnn_input_shape)
# satellite_input.append(satellite_placeholder)

satellite_input_2 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Polonnaruwa/Seasonal_Images_Polonnaruwa", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_2)
# satellite_input.append(satellite_placeholder)

satellite_input_3 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Mannar/Seasonal_Images_Mannar", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_3)
# satellite_input.append(satellite_placeholder)

satellite_input_4 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Trincomalee/Seasonal_Images_Trincomalee", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_4)
# satellite_input.append(satellite_placeholder)

satellite_input_5 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Badulla/Seasonal_Images_Badulla", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_5)

# Yeild Data

In [ ]:
# Yeild data

import pandas as pd
import numpy as np

# Load yield data
yield_data = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_hambantota_yield.csv')
yield_data1 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_polonnaruwa_yield.csv')
yield_data2 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_mannar_yield.csv')
yield_data3 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_trincomalee_yield.csv')
yield_data4 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_badulla_yield.csv')


def create_y_target(yield_data):
    # Ensure the data is sorted by Year and District (if needed)
    yield_data = yield_data.sort_values(by=['Year', 'District'])

    # Extract yield values and map to seasons
    years = ["2019", "2020", "2021", "2022", "2023"]
    seasons = ["Yala", "Maha"]

    Y_target = []

    for year in years:
        # Filter data for the current year
        year_data = yield_data[yield_data['Year'] == int(year)]

        # Extract Yala and Maha yields
        yala_yield = year_data.iloc[0]['Yield']
        maha_yield = year_data.iloc[1]['Yield']

        # Append to Y_target
        Y_target.append(yala_yield)
        Y_target.append(maha_yield)

    # Convert Y_target to numpy array
    # Y_target.append(0)
    Y_target = np.array(Y_target)
    return Y_target

Y_target1 = create_y_target(yield_data)
Y_target2 = create_y_target(yield_data1)
Y_target3 = create_y_target(yield_data2)
Y_target4 = create_y_target(yield_data3)
Y_target5 = create_y_target(yield_data4)

Y_target = np.concatenate((Y_target1, Y_target2))
Y_target = np.concatenate((Y_target, Y_target3))
Y_target = np.concatenate((Y_target, Y_target4))
Y_target = np.concatenate((Y_target, Y_target5))
# Y_target = Y_target[:-1]
Y_target = Y_target*1000*0.404686
print(Y_target)

[2679.82203617 2317.28017421 2550.45923003 2422.29913044 2380.82762505
 2352.72251743 1606.31745511 1866.58856121 2212.76128725 2058.60540505
 2053.71162345 2230.22648242 2186.97648    2122.35815189 2112.55637329
 2201.74433331 1255.6830813  1735.42527711 1787.22812632 1990.36186779
 2068.85708158 2232.03103112 1953.46562941 2174.56192191 2115.81724283
 1995.49921775 1077.4536236  1385.92372624 1415.31802951 1953.29873933
 1716.20971907 2014.52469356 1770.34622289 2132.01870763 1719.73780966
 2131.40610003 1077.31227568 1629.36273847 1270.06261088 1930.70878155
 1905.33075576 2071.82767269 1782.82797261 2012.58983111 1966.44306809
 2059.12138496 1224.88532415 1355.41832843 1481.85811471 1842.87500529]


# Ground Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the ground data for different locations
ground_data_paths = [
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Polonnaruwa.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Mannar.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Trincomalee.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Badulla.csv"
]

ground_dfs = [pd.read_csv(path) for path in ground_data_paths]

# Extract relevant columns (adjust as per your needs)
features = ['temperature_2m_max (°C)', 'temperature_2m_min (°C)', 'precipitation_sum (mm)', 'wind_speed_10m_max (km/h)']  # Example features

# Normalize the data
scaler = MinMaxScaler()
# for df in ground_dfs:
#     df[features] = scaler.fit_transform(df[features])

# Define the months for each season (use months 1 and 2 from the next year for Maha)
season_months = {
    "Yala": [3, 4, 5, 6, 7, 8],
    "Maha": [9, 10, 11, 12, 1, 2]
}

# Prepare ground data, moving months 1 and 2 to the previous year's Maha season
def prepare_ground_data(df, years, seasons):
    ground_data = []
    for year in years:
        for season in seasons:
            if season == 'Maha':
                # Maha includes months from both the current and next year
                maha_df = df[((df['year'] == int(year)) & (df['month'].isin([9, 10, 11, 12]))) |
                             ((df['year'] == int(year) + 1) & (df['month'].isin([1, 2])))]
            else:
                # Yala includes months within the same year
                maha_df = df[(df['year'] == int(year)) & (df['month'].isin([3, 4, 5, 6, 7, 8]))]

            # Extract relevant features for the season
            ground_data.append(maha_df[features].values)
    return ground_data

# Prepare the ground data for all years and seasons
years = [2019, 2020, 2021, 2022, 2023]  # List of years to process
seasons = ["Yala", "Maha"]

# Initialize an empty list for ground data
ground_input = []

# Prepare the ground data for each location
for df in ground_dfs:
    ground_input.extend(prepare_ground_data(df, years, seasons))


# Check the shapes
for i, season_data in enumerate(satellite_input):
    print(f"Season {i+1}: Satellite input shape = {season_data.shape}, Ground input shape = {ground_input[i].shape}")

ground_input[-1]


Season 1: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 2: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 3: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 4: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 5: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 6: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 7: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 8: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 9: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 10: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 11: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 12: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 13: Satellite input shape = (6

array([[28.1       , 20.80666667,  6.69333333,  8.27      ],
       [26.67741935, 20.08709677, 12.59032258,  7.62580645],
       [26.14666667, 19.86      , 15.75333333,  7.7       ],
       [25.34193548, 20.21612903, 13.34516129,  9.51290323],
       [25.0483871 , 18.77419355,  8.0516129 , 10.67096774],
       [26.34827586, 18.85172414,  3.66551724, 11.26896552]])

# Check Cardinality

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

X_satellite = np.array(satellite_input)  # Shape: (10, T, 128, 128, 4)
X_ground = np.array(ground_input)        # Shape: (10, T, 4)

T = None  # Let T be variable because the input sequence length varies


# model = create_stacked_lstm_model(cnn_input_shape, ground_input_shape, T)

# Split satellite data
# X_satellite_train, X_satellite_test, X_ground_train, X_ground_test, Y_train, Y_test = train_test_split(
#     X_satellite, X_ground, Y_target, test_size=0.2, random_state=42
# )

split_point = len(X_satellite) - 10

X_satellite_train = (X_satellite[:split_point])
X_ground_train = (X_ground[:split_point])
Y_train = (Y_target[:split_point])

X_satellite_test = (X_satellite[split_point:])
X_ground_test = (X_ground[split_point:])
Y_test = (Y_target[split_point:])
Y_test


array([1905.33075576, 2071.82767269, 1782.82797261, 2012.58983111,
       1966.44306809, 2059.12138496, 1224.88532415, 1355.41832843,
       1481.85811471, 1842.87500529])

In [ ]:


# Print shapes for satellite input
print("Satellite Input (X_satellite):")
print(f"Shape of full dataset: {X_satellite.shape}")
print(f"Shape of training data: {X_satellite_train.shape}")
print(f"Shape of test data: {X_satellite_test.shape}")

# Uncomment and use if you include ground data
print("\nGround Input (X_ground):")
print(f"Shape of full dataset: {X_ground.shape}")
print(f"Shape of training data: {X_ground_train.shape}")
print(f"Shape of test data: {X_ground_test.shape}")

# Print shapes for target data
print("\nTarget Output (Y_target):")
print(f"Shape of full dataset: {Y_target.shape}")
print(f"Shape of training data: {Y_train.shape}")
print(f"Shape of test data: {Y_test.shape}")


Satellite Input (X_satellite):
Shape of full dataset: (50, 6, 256, 256, 4)
Shape of training data: (40, 6, 256, 256, 4)
Shape of test data: (10, 6, 256, 256, 4)

Ground Input (X_ground):
Shape of full dataset: (50, 6, 4)
Shape of training data: (40, 6, 4)
Shape of test data: (10, 6, 4)

Target Output (Y_target):
Shape of full dataset: (50,)
Shape of training data: (40,)
Shape of test data: (10,)


# Model Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import (
    Input,
    TimeDistributed,
    Conv2D,
    MaxPooling2D,
    Dense,
    Flatten,
    concatenate,
    Dropout,
    UpSampling2D,
)
import numpy as np
from tensorflow.keras.preprocessing.image import smart_resize

# Load the trained model
trained_model_path = "/content/drive/MyDrive/models/model2_datacleared.keras"
trained_model = load_model(trained_model_path)

# Redefine the pretrained model input layer
def modify_model(trained_model, new_input_shape):
    # Extract the layers and weights from the trained model
    trained_layers = trained_model.layers[:-1]  # Exclude the output layer
    pretrained_output_layer = trained_model.layers[-1]  # Output layer

    # Create a new input layer
    new_input = Input(shape=new_input_shape, name="updated_satellite_input")

    # Rebuild the model using the pretrained layers
    x = new_input
    for layer in trained_layers[1:]:  # Skip the original input layer
        x = layer(x)

    # Add the original output layer
    output = pretrained_output_layer(x)
    return Model(inputs=new_input, outputs=output)

# Modify the model to accept new input shape
satellite_model = modify_model(trained_model, new_input_shape=(6, 33, 33, 4))

# Create a new branch for ground data
ground_input = Input(shape=(6, 4), name="ground_input")
ground_flatten = TimeDistributed(Dense(16, activation="relu"))(ground_input)
ground_flatten = Flatten()(ground_flatten)
# Adjust the shape of ground_flatten to match satellite_features
ground_flatten = Dense(256, activation="relu")(ground_flatten) # Assuming satellite_features has shape (None, 256) after Flatten

# Combine satellite and ground data
satellite_input = Input(shape=(6, 256, 256, 4), name="satellite_input")

# Convolutional branch for satellite data
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(satellite_input)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(32, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)

# Adjust the upsampling to match the expected input shape of (6, 33, 33, 4)
x = TimeDistributed(Conv2D(4, (1, 1), activation="relu", padding="same"))(x)  # Reduce channels to 4
x = TimeDistributed(tf.keras.layers.Lambda(lambda img: smart_resize(img, (33, 33))))(x)  # Resize to (33, 33, 4)


# Output shape is now (None, 6, 33, 33, 4)
satellite_features = satellite_model(x)
# Flatten the satellite features to match ground_flatten
satellite_features = Flatten()(satellite_features)

# Combine both branches
combined = concatenate([satellite_features, ground_flatten])

# Add fully connected layers for prediction
dense = Dense(128, activation="relu")(combined)
dense = Dropout(0.5)(dense)
output = Dense(1, activation="linear", name="output")(dense)

# Create the final model
final_model = Model(inputs=[satellite_input, ground_input], outputs=output)
final_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Model Summary
final_model.summary()


# Fine-tune the combined model
history = final_model.fit(
    [X_satellite_train, X_ground_train],
    Y_train,
    validation_data=([X_satellite_test, X_ground_test], Y_test),
    batch_size=8,
    epochs=20
)

Model: "functional_60"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ satellite_input           │ (None, 6, 256, 256, 4) │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_31       │ (None, 6, 256, 256,    │          2,368 │ satellite_input[0][0]  │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_32       │ (None, 6, 128, 128,    │              0 │ time_distributed_31[0… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_33       │ (None, 6, 128, 128,    │         36,928 │ time_distributed_32[0… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_34       │ (None, 6, 64, 64, 64)  │              0 │ time_distributed_33[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_35       │ (None, 6, 64, 64, 32)  │         18,464 │ time_distributed_34[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_36       │ (None, 6, 32, 32, 32)  │              0 │ time_distributed_35[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_37       │ (None, 6, 32, 32, 4)   │            132 │ time_distributed_36[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ground_input (InputLayer) │ (None, 6, 4)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_38       │ (None, 6, 33, 33, 4)   │              0 │ time_distributed_37[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_30       │ (None, 6, 16)          │             80 │ ground_input[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_59             │ (None, 6, 1)           │      1,150,945 │ time_distributed_38[0… │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_6 (Flatten)       │ (None, 96)             │              0 │ time_distributed_30[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_7 (Flatten)       │ (None, 6)              │              0 │ functional_59[0][0]    │
├──────────────────────

 Total params: 1,267,542 (4.84 MB)

 Trainable params: 1,267,350 (4.83 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 148s 27s/step - loss: 3182388.0000 - mae: 1725.5874 - val_loss: 964544.6250 - val_mae: 938.0209
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 152s 27s/step - loss: 889630.6250 - mae: 805.3510 - val_loss: 87963.3984 - val_mae: 272.2940
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 133s 27s/step - loss: 784793.0625 - mae: 772.7896 - val_loss: 219267.7500 - val_mae: 411.0649
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 142s 27s/step - loss: 513910.1875 - mae: 603.4747 - val_loss: 287145.8125 - val_mae: 469.4076
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 142s 27s/step - loss: 436977.7812 - mae: 533.6147 - val_loss: 142480.0312 - val_mae: 344.8459
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 153s 30s/step - loss: 276661.5625 - mae: 428.9025 - val_loss: 84135.1250 - val_mae: 248.0369
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 192s 27s/step - loss: 263847.8125 - mae: 422.2697 - val_loss: 97214.6094 - val_mae: 232.6550
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 141s 27s/step - loss: 275802.7500 - mae: 432.3938 - val

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import (
    Input,
    TimeDistributed,
    Conv2D,
    MaxPooling2D,
    Dense,
    Flatten,
    concatenate,
    Dropout,
    UpSampling2D,
)
import numpy as np
from tensorflow.keras.preprocessing.image import smart_resize

# Load the trained model
trained_model_path = "/content/drive/MyDrive/models/model2_datacleared.keras"
trained_model = load_model(trained_model_path)

# Redefine the pretrained model input layer
def modify_model(trained_model, new_input_shape):
    # Extract the layers and weights from the trained model
    trained_layers = trained_model.layers[:-1]  # Exclude the output layer
    pretrained_output_layer = trained_model.layers[-1]  # Output layer

    # Create a new input layer
    new_input = Input(shape=new_input_shape, name="updated_satellite_input")

    # Rebuild the model using the pretrained layers
    x = new_input
    for layer in trained_layers[1:]:  # Skip the original input layer
        x = layer(x)

    # Add the original output layer
    output = pretrained_output_layer(x)
    return Model(inputs=new_input, outputs=output)

# Modify the model to accept new input shape
satellite_model = modify_model(trained_model, new_input_shape=(6, 33, 33, 4))

# Create a new branch for ground data
ground_input = Input(shape=(6, 4), name="ground_input")
ground_flatten = TimeDistributed(Dense(16, activation="relu"))(ground_input)
ground_flatten = Flatten()(ground_flatten)
ground_flatten = Dense(256, activation="relu")(ground_flatten)  # Adjust shape to match satellite features

# Convolutional branch for satellite data
satellite_input = Input(shape=(6, 256, 256, 4), name="satellite_input")
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(satellite_input)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(32, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)

# Adjust the upsampling to match the expected input shape of (6, 33, 33, 4)
x = TimeDistributed(Conv2D(4, (1, 1), activation="relu", padding="same"))(x)  # Reduce channels to 4
x = TimeDistributed(tf.keras.layers.Lambda(lambda img: smart_resize(img, (33, 33))))(x)  # Resize to (33, 33, 4)

# Extract features using the satellite_model
satellite_features = satellite_model(x)  # Shape: (None, 6, 1)
satellite_features = TimeDistributed(Flatten())(satellite_features)  # Shape: (None, 6, 1)
satellite_features = Flatten()(satellite_features)  # Shape: (None, 6 * 1)

# Combine satellite and ground data
combined = concatenate([satellite_features, ground_flatten])

# Add fully connected layers for prediction
dense = Dense(128, activation="relu")(combined)
dense = Dropout(0.5)(dense)
output = Dense(1, activation="linear", name="output")(dense)

# Create the final model
final_model = Model(inputs=[satellite_input, ground_input], outputs=output)
final_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Model Summary
final_model.summary()



# Fine-tune the combined model
history = final_model.fit(
    [X_satellite_train, X_ground_train],
    Y_train,
    validation_data=([X_satellite_test, X_ground_test], Y_test),
    batch_size=8,
    epochs=20
)


Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ satellite_input           │ (None, 6, 256, 256, 4) │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_12       │ (None, 6, 256, 256,    │          2,368 │ satellite_input[0][0]  │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_13       │ (None, 6, 128, 128,    │              0 │ time_distributed_12[0… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_14       │ (None, 6, 128, 128,    │         36,928 │ time_distributed_13[0… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_15       │ (None, 6, 64, 64, 64)  │              0 │ time_distributed_14[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_16       │ (None, 6, 64, 64, 32)  │         18,464 │ time_distributed_15[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_17       │ (None, 6, 32, 32, 32)  │              0 │ time_distributed_16[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_18       │ (None, 6, 32, 32, 4)   │            132 │ time_distributed_17[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_19       │ (None, 6, 33, 33, 4)   │              0 │ time_distributed_18[0… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ground_input (InputLayer) │ (None, 6, 4)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_19             │ (None, 6, 1)           │      1,150,945 │ time_distributed_19[0… │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_11       │ (None, 6, 16)          │             80 │ ground_input[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_20       │ (None, 6, 1)           │              0 │ functional_19[0][0]    │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)  

 Total params: 1,267,542 (4.84 MB)

 Trainable params: 1,267,350 (4.83 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 152s 27s/step - loss: 3924042.0000 - mae: 1924.8549 - val_loss: 2672428.5000 - val_mae: 1608.7213
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 142s 27s/step - loss: 3349807.0000 - mae: 1788.1646 - val_loss: 1906037.6250 - val_mae: 1349.7909
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 194s 27s/step - loss: 2314843.2500 - mae: 1435.8743 - val_loss: 708776.7500 - val_mae: 790.5999
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 141s 27s/step - loss: 795911.0625 - mae: 729.0956 - val_loss: 226520.0312 - val_mae: 417.7978
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 152s 29s/step - loss: 469905.8125 - mae: 573.0808 - val_loss: 350471.1875 - val_mae: 521.8925
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 139s 27s/step - loss: 397925.6562 - mae: 519.7300 - val_loss: 473657.5000 - val_mae: 624.9612
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 136s 27s/step - loss: 434947.9375 - mae: 563.0287 - val_loss: 308996.3438 - val_mae: 488.6459
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 141s 27s/step - loss: 260268.6406 - mae: 427

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    TimeDistributed,
    Conv2D,
    MaxPooling2D,
    Dense,
    Flatten,
    concatenate,
    Dropout,
    LSTM,
    Lambda,
    BatchNormalization,
)
from tensorflow.keras.preprocessing.image import smart_resize

def modify_model(trained_model, new_input_shape):
    # Extract the layers and weights from the trained model
    trained_layers = trained_model.layers[:-1]  # Exclude the output layer
    pretrained_output_layer = trained_model.layers[-1]  # Output layer

    # Create a new input layer
    new_input = Input(shape=new_input_shape, name="updated_satellite_input")

    # Rebuild the model using the pretrained layers
    x = new_input
    for layer in trained_layers[1:]:  # Skip the original input layer
        x = layer(x)

    # Add the original output layer
    output = pretrained_output_layer(x)
    return Model(inputs=new_input, outputs=output)
# Load and modify the pretrained model
trained_model = tf.keras.models.load_model(trained_model_path)
satellite_model = modify_model(trained_model, new_input_shape=(33, 33, 4))

# Ground data branch
ground_input = Input(shape=(6, 4), name="ground_input")
ground_features = TimeDistributed(Dense(64, activation="relu"))(ground_input)
ground_features = LSTM(128, return_sequences=False)(ground_features)
ground_features = Dense(256, activation="relu")(ground_features)

# Satellite data branch
satellite_input = Input(shape=(6, 256, 256, 4), name="satellite_input")
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(satellite_input)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(32, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(4, (1, 1), activation="relu", padding="same"))(x)

# Resize to match pretrained model input
x = TimeDistributed(Lambda(lambda img: smart_resize(img, (33, 33))))(x)

# Process each timestep through the satellite model
satellite_features = TimeDistributed(satellite_model)(x)
satellite_features = TimeDistributed(Flatten())(satellite_features)
satellite_features = LSTM(128, return_sequences=False)(satellite_features)

# Combine features and create output layers
combined = concatenate([satellite_features, ground_features])
dense = Dense(256, activation="relu")(combined)
dense = Dropout(0.4)(dense)
dense = Dense(128, activation="relu")(dense)
output = Dense(1, activation="linear", name="output")(dense)

# Create and compile model
final_model = Model(inputs=[satellite_input, ground_input], outputs=output)
final_model.compile(optimizer="adam", loss="mse", metrics=["mae"])
# Model Summary
final_model.summary()

# Fine-tune the combined model
history = final_model.fit(
    [X_satellite_train, X_ground_train],
    Y_train,
    validation_data=([X_satellite_test, X_ground_test], Y_test),
    batch_size=8,
    epochs=10
)


ValueError: Exception encountered when calling TimeDistributed.call().

[1mInput 0 of layer "functional_96" is incompatible with the layer: expected shape=(None, 33, 33, 4), found shape=(None, 33, 4)[0m

Arguments received by TimeDistributed.call():
  • args=('<KerasTensor shape=(None, 33, 33, 4), dtype=float32, sparse=False, name=updated_satellite_input>',)
  • kwargs={'mask': 'None'}

# Saving & Validating

In [ ]:
# Save the trained model
final_model.save('/content/drive/MyDrive/models/model5_ground_finetune.keras')

print("Model saved successfully!")

Model saved successfully!


In [ ]:
# Evaluate the model on the test data
test_loss, test_mae = final_model.evaluate(
    [X_satellite_test, X_ground_test],
    Y_test,
    batch_size=8
)
print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 801ms/step - loss: 335174.1250 - mae: 501.1043
Test Loss: 342121.9375, Test MAE: 511.1561


In [ ]:
model = load_model('/content/drive/MyDrive/models/model4_ground_finetune.keras')

In [ ]:
# Make predictions on the test set
predictions = final_model.predict([X_satellite_test, X_ground_test], batch_size=8)

# Print some predictions vs. actual values
for i in range(10):  # Print the first 5 predictions
    print(f"Prediction: {predictions[i][0]:.4f}, Actual: {Y_test[i]:.4f}")


1/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step
Prediction: 2278.3943, Actual: 1905.3308
Prediction: 2297.1323, Actual: 2071.8277
Prediction: 2273.7490, Actual: 1782.8280
Prediction: 2308.3433, Actual: 2012.5898
Prediction: 2272.5447, Actual: 1966.4431
Prediction: 2263.8279, Actual: 2059.1214
Prediction: 2274.3052, Actual: 1224.8853
Prediction: 2258.1538, Actual: 1355.4183
Prediction: 2275.9714, Actual: 1481.8581
Prediction: 2312.3164, Actual: 1842.8750


In [ ]:
# Calculate MAPE
# Y_to= Y_test/2471
mape = np.mean(np.abs((Y_test - predictions.flatten()) / Y_test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


Mean Absolute Percentage Error (MAPE): 32.95%


# Full Architecture Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import (
    Input,
    TimeDistributed,
    Conv2D,
    MaxPooling2D,
    Dense,
    Flatten,
    concatenate,
    Dropout,
    UpSampling2D,
    Reshape,
    LSTM,
)
import numpy as np
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras.optimizers import Adam

# Load the trained model
trained_model_path = "/content/drive/MyDrive/models/model2_datacleared.keras"
trained_model = load_model(trained_model_path)

# Redefine the pretrained model input layer
def modify_model(trained_model, new_input_shape):
    # Extract the layers and weights from the trained model
    trained_layers = trained_model.layers[:-1]  # Exclude the output layer
    pretrained_output_layer = trained_model.layers[-1]  # Output layer

    # Create a new input layer
    new_input = Input(shape=new_input_shape, name="updated_satellite_input")

    # Rebuild the model using the pretrained layers
    x = new_input
    for layer in trained_layers[1:]:  # Skip the original input layer
        x = layer(x)

    # Add the original output layer
    output = pretrained_output_layer(x)
    return Model(inputs=new_input, outputs=output)

# Modify the model to accept new input shape
satellite_model = modify_model(trained_model, new_input_shape=(6, 33, 33, 4))

# Create a new branch for ground data
ground_input = Input(shape=(6, 4), name="ground_input")
ground_flatten = TimeDistributed(Dense(16, activation="relu"))(ground_input)
ground_flatten = Flatten()(ground_flatten)
# Adjust the shape of ground_flatten to match satellite_features
ground_flatten = Dense(256, activation="relu")(ground_flatten) # Assuming satellite_features has shape (None, 256) after Flatten

# Combine satellite and ground data
satellite_input = Input(shape=(6, 256, 256, 4), name="satellite_input")

# Convolutional branch for satellite data
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(satellite_input)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(64, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)
x = TimeDistributed(Conv2D(32, (3, 3), activation="relu", padding="same"))(x)
x = TimeDistributed(MaxPooling2D((2, 2), padding="same"))(x)

# Adjust the upsampling to match the expected input shape of (6, 33, 33, 4)
x = TimeDistributed(Conv2D(4, (1, 1), activation="relu", padding="same"))(x)  # Reduce channels to 4
x = TimeDistributed(tf.keras.layers.Lambda(lambda img: smart_resize(img, (33, 33))))(x)  # Resize to (33, 33, 4)


# Output shape is now (None, 6, 33, 33, 4)
satellite_features = satellite_model(x)
# Flatten the satellite features to match ground_flatten
satellite_features = Flatten()(satellite_features)

# Combine both branches
combined = concatenate([satellite_features, ground_flatten])


# Reshape the combined features for LSTM processing
combined_reshaped = Reshape((2, 131))(combined)  # Reshape to (6, feature_dim)

# Add LSTM layers
lstm_out = LSTM(256, return_sequences=True, activation="relu")(combined_reshaped)
lstm_out = LSTM(128, return_sequences=False, activation="relu")(lstm_out)  # Final LSTM outputs a single sequence

# Add fully connected layers for prediction
dense = Dense(128, activation="relu")(lstm_out)
dense = Dropout(0.5)(dense)
output = Dense(1, activation="linear", name="output")(dense)

# Create the final model
final_model = Model(inputs=[satellite_input, ground_input], outputs=output)

optimizer = Adam(learning_rate=0.001)

final_model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

# Model Summary
final_model.summary()

# Fine-tune the combined model
history = final_model.fit(
    [X_satellite_train, X_ground_train],
    Y_train,
    validation_data=([X_satellite_test, X_ground_test], Y_test),
    batch_size=8,
    epochs=25,
)

Model: "functional_153"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ satellite_input           │ (None, 6, 256, 256, 4) │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_127      │ (None, 6, 256, 256,    │          2,368 │ satellite_input[0][0]  │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_128      │ (None, 6, 128, 128,    │              0 │ time_distributed_127[… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_129      │ (None, 6, 128, 128,    │         36,928 │ time_distributed_128[… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_130      │ (None, 6, 64, 64, 64)  │              0 │ time_distributed_129[… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_131      │ (None, 6, 64, 64, 32)  │         18,464 │ time_distributed_130[… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_132      │ (None, 6, 32, 32, 32)  │              0 │ time_distributed_131[… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_133      │ (None, 6, 32, 32, 4)   │            132 │ time_distributed_132[… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ground_input (InputLayer) │ (None, 6, 4)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_134      │ (None, 6, 33, 33, 4)   │              0 │ time_distributed_133[… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_126      │ (None, 6, 16)          │             80 │ ground_input[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_152            │ (None, 6, 1)           │      1,150,945 │ time_distributed_134[… │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_28 (Flatten)      │ (None, 96)             │              0 │ time_distributed_126[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_29 (Flatten)      │ (None, 6)              │              0 │ functional_152[0][0]   │
├──────────────────────

 Total params: 1,844,822 (7.04 MB)

 Trainable params: 1,844,630 (7.04 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 137s 24s/step - loss: 4074717.5000 - mae: 1986.9303 - val_loss: 2895364.7500 - val_mae: 1676.4362
Epoch 2/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 137s 23s/step - loss: 3405902.2500 - mae: 1810.3761 - val_loss: 2210398.0000 - val_mae: 1457.9642
Epoch 3/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 143s 24s/step - loss: 1635788.5000 - mae: 1204.4368 - val_loss: 441370.9062 - val_mae: 595.7873
Epoch 4/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 118s 24s/step - loss: 898321.1875 - mae: 708.3566 - val_loss: 386760.0938 - val_mae: 549.9286
Epoch 5/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 140s 24s/step - loss: 341039.6562 - mae: 498.2892 - val_loss: 803191.8750 - val_mae: 850.1622
Epoch 6/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 116s 23s/step - loss: 456672.5000 - mae: 602.1144 - val_loss: 409789.3438 - val_mae: 570.5543
Epoch 7/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 144s 24s/step - loss: 320484.0000 - mae: 448.6880 - val_loss: 84845.0234 - val_mae: 253.4106
Epoch 8/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 139s 24s/step - loss: 451829.5000 - mae: 550.

In [ ]:
# Make predictions on the test set
predictions = final_model.predict([X_satellite_test, X_ground_test], batch_size=8)

# Print some predictions vs. actual values
for i in range(10):  # Print the first 5 predictions
    print(f"Prediction: {predictions[i][0]:.4f}, Actual: {Y_test[i]:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step
Prediction: 1866.3010, Actual: 1905.3308
Prediction: 1869.3412, Actual: 2071.8277
Prediction: 1866.4971, Actual: 1782.8280
Prediction: 1871.5635, Actual: 2012.5898
Prediction: 1867.6073, Actual: 1966.4431
Prediction: 1867.3632, Actual: 2059.1214
Prediction: 1867.7778, Actual: 1224.8853
Prediction: 1867.5778, Actual: 1355.4183
Prediction: 1867.8370, Actual: 1481.8581
Prediction: 1869.7255, Actual: 1842.8750


In [ ]:
# Calculate MAPE
# Y_to= Y_test/2471
mape = np.mean(np.abs((Y_test - predictions.flatten()) / Y_test)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Mean Absolute Percentage Error (MAPE): 15.56%


In [ ]:
final_model.save('/content/drive/MyDrive/models/bestmodel_15mape.keras')